In [1]:
import tensorflow as tf
import numpy as np
import cv2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
import matplotlib.pyplot as plt
import itertools

/Users/hiroyukiqaq/.virtualenvs/cv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [17]:
# input_fn = 'presentation_material/Selfies/lebron_GREY.jpeg'
# test_img = cv2.imread(input_fn)
# test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
# test_img.shape

(48, 48)

In [18]:
# read the data set
def csv_reading(f_name):
    with open(f_name, 'r') as data:
        temp = pd.read_csv(data)
    temp = temp.values.tolist()
    for i in range(len(temp)):
        temp[i][1] = temp[i][1].split(' ')
    images = [item[1] for item in temp]
    for i in range(len(images)):
        images[i] = np.asarray([int(x) for x in images[i]])
        images[i] = images[i].reshape([48, 48, 1])
    labels = [item[0] for item in temp]
    return np.asarray(images), labels

images, labels = csv_reading('fer2013.csv')
test_labels = labels[32298:]
test_img = images[32298:]
predict_labels = []

In [19]:
with tf.Session(graph=tf.Graph()) as sess1:
    tf.saved_model.loader.load(
        sess1,
        [tf.saved_model.tag_constants.SERVING],
        "DeepCNN_Softmax"
    )

    prediction = sess1.run(
        'Softmax:0',
        feed_dict={
            'Placeholder:0': test_img.reshape([-1, 48, 48, 1])
        }
    )
    result = np.argmax(prediction, axis=1)
    print(np.argmax(prediction, axis=1))
    

INFO:tensorflow:Restoring parameters from b'DeepCNN_Softmax/variables/variables'
[6]


In [7]:
target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print(classification_report(test_labels, result, target_names=target_names))

              precision    recall  f1-score   support

       Angry       0.55      0.48      0.52       491
     Disgust       0.53      0.42      0.47        55
        Fear       0.48      0.29      0.36       528
       Happy       0.75      0.85      0.80       879
         Sad       0.41      0.57      0.48       594
    Surprise       0.74      0.68      0.71       416
     Neutral       0.55      0.53      0.54       626

   micro avg       0.59      0.59      0.59      3589
   macro avg       0.57      0.55      0.55      3589
weighted avg       0.59      0.59      0.58      3589



In [29]:
# plot the confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Oranges):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plt.figure()
plot_confusion_matrix(confusion_matrix(test_labels, result), classes=target_names,
                      title='Confusion matrix, with normalization', normalize=True)
plt.show()

In [ ]:
# with tf.Session(graph=tf.Graph()) as sess2:
#     tf.saved_model.loader.load(
#         sess2,
#         [tf.saved_model.tag_constants.SERVING],
#         "DeepCNN_Loss_Accuracy"
#     )

#     loss1, accuracy1 = sess.run(
#         ['Mean:0','Mean_1:0'],
#         feed_dict={
#             'Placeholder:0': test_img,
#             'Placeholder_1:0': test_labels})
    

#     print(loss1, accuracy1)

In [32]:
# np.savez('fer2013_data_set.npz', images = images, labels = labels)